In [1]:
# create pybash macro
# https://stackoverflow.com/a/67029719/7782
from IPython import get_ipython
from IPython.core.magic import register_cell_magic

ipython = get_ipython()

@register_cell_magic
def pybash(line, cell):
    ipython.run_cell_magic('bash', '', cell.format(**globals()))

In [2]:
from pathlib import Path as P

import pandas as pd
import numpy as np
import geopandas as gpd
import folium



In [ ]:
%%bash

# use gpq to describe a file
cd ~/data/iSample/2024_07_10_15_51_57/ 

gpq describe isamples_export_2024_07_10_15_51_57_geo.parquet


In [ ]:
parquet_path = P.home() / 'data/iSample/2024_07_10_15_51_57/isamples_export_2024_07_10_15_51_57_geo.parquet'
# parquet_path = "cities.geoparquet"

# Step 1: Read the GeoParquet file
gdf = gpd.read_parquet(str(parquet_path))

# Step 2: Number of Records
num_records = len(gdf)
print(f"Number of records: {num_records}")

# Step 3: Bounding Box
bbox = gdf.total_bounds  # This gives [minx, miny, maxx, maxy] of the entire GeoDataFrame
print(f"Bounding Box: {bbox}")

# Step 4: Centroid
# This calculates the centroid of the combined geometries, not the average of centroids
combined_centroid = gdf.unary_union.centroid
print(f"Centroid: {combined_centroid.x}, {combined_centroid.y}")

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt

# Create a figure and axis
fig, ax = plt.subplots(figsize=(10, 10))

# Plot the points
gdf.plot(ax=ax, marker='o', color='blue', markersize=5, alpha=0.5)

# Remove axis
ax.axis('off')

# Add a title
plt.title('Static Representation of Points')

# Show the plot
plt.show()

# Optionally, save the plot
# plt.savefig('points_plot.png', dpi=300, bbox_inches='tight')

# OpenContext 2025.02

In [ ]:
import geopandas as gpd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


parquet_url = "https://storage.cloud.google.com/opencontext-parquet/oc_all_assertions.parquet"
parquet_path = P.home() / 'data/iSample/opencontext/oc_all_assertions.parquet'


df = pd.read_parquet(str(parquet_path))

In [ ]:
# use duckdb to read parquet
import duckdb

# Read the Parquet file into a DuckDB table
con = duckdb.connect()

# describe oc_all_assertions.parquet
# DESCRIBE SELECT * FROM 'oc_all_assertions.parquet';
df = con.execute(f"DESCRIBE SELECT * FROM '{parquet_path}'").fetchdf()

df.head()

In [ ]:
import geopandas as gpd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path as P

# isamples_oc_test.parquet isn't a geoparquet file
# see whether we can convert it to a geoparquet file

parquet_path = P.home() / 'data/iSample/opencontext/isamples_oc_test.parquet'
df = pd.read_parquet(str(parquet_path))



In [ ]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

def create_geometry(row):
    # For now, all geometries are points, but we'll keep the structure
    # to make it easy to add polygon support later
    geom_type = row['item__geometry_type']
    
    # Default to using lat/long points for all types currently
    if pd.notna(row['item__latitude']) and pd.notna(row['item__longitude']):
        return Point(row['item__longitude'], row['item__latitude'])
    else:
        return None
    
    # Future implementation for other geometry types:
    # elif geom_type == 'Polygon':
    #     # TODO: Implement polygon creation
    #     pass
    # elif geom_type == 'MultiPolygon':
    #     # TODO: Implement multipolygon creation
    #     pass


# Create geometry column
df['geometry'] = df.apply(create_geometry, axis=1)

# Remove rows with null geometries if any exist
df = df.dropna(subset=['geometry'])

# Convert to GeoDataFrame
gdf = gpd.GeoDataFrame(df, geometry='geometry')

# Set CRS to WGS 84 since we're using lat/long
gdf.set_crs(epsg=4326, inplace=True)

# Write to GeoParquet
gdf.to_parquet(P.home() / 'data/iSample/opencontext' / 'output_geoparquet.parquet')

In [ ]:

# Step 2: Number of Records
num_records = len(gdf)
print(f"Number of records: {num_records}")

# Step 3: Bounding Box
bbox = gdf.total_bounds  # This gives [minx, miny, maxx, maxy] of the entire GeoDataFrame
print(f"Bounding Box: {bbox}")

# Step 4: Centroid
# This calculates the centroid of the combined geometries, not the average of centroids
combined_centroid = gdf.unary_union.centroid
print(f"Centroid: {combined_centroid.x}, {combined_centroid.y}")

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt

# Create a figure and axis
fig, ax = plt.subplots(figsize=(10, 10))

# Plot the points
gdf.plot(ax=ax, marker='o', color='blue', markersize=5, alpha=0.5)

# Remove axis
ax.axis('off')

# Add a title
plt.title('Static Representation of Points')

# Show the plot
plt.show()

# Optionally, save the plot
# plt.savefig('points_plot.png', dpi=300, bbox_inches='tight')

In [ ]:
import pandas as pd
import duckdb
from pathlib import Path as P
import geopandas as gpd

def setup_duckdb():
    """Set up DuckDB with spatial extension"""
    con = duckdb.connect()
    con.install_extension('spatial')
    con.load_extension('spatial')
    return con

def create_spatial_table(con, parquet_file):
    """Create a table with spatial data from parquet file"""
    df = pd.read_parquet(parquet_file)
    
    con.execute("""
        CREATE TABLE archaeological_sites AS 
        SELECT 
            uuid,
            label,
            item__latitude as lat,
            item__longitude as lon,
            item__earliest as earliest_date,
            item__latest as latest_date,
            path_to___2 as country,
            ST_Point(item__longitude, item__latitude) as geom
        FROM df
        WHERE item__latitude IS NOT NULL 
        AND item__longitude IS NOT NULL
    """)
    
    # Print quick summary
    count = con.execute("SELECT COUNT(*) FROM archaeological_sites").fetchone()[0]
    print(f"Created table with {count} sites")

def quick_stats(con):
    """Get quick statistics about the dataset"""
    
    print("=== Quick Dataset Statistics ===")
    
    # Sites by country (top 5)
    print("\nTop 5 countries by number of sites:")
    result = con.execute("""
        SELECT 
            country,
            COUNT(*) as site_count
        FROM archaeological_sites
        GROUP BY country
        ORDER BY site_count DESC
        LIMIT 5
    """).df()
    print(result)
    
    # Time range
    print("\nTemporal range of sites:")
    result = con.execute("""
        SELECT 
            MIN(earliest_date) as earliest_date,
            MAX(latest_date) as latest_date
        FROM archaeological_sites
        WHERE earliest_date IS NOT NULL 
        AND latest_date IS NOT NULL
    """).df()
    print(result)
    
    # Geographic extent
    print("\nGeographic extent:")
    result = con.execute("""
        SELECT 
            ROUND(MIN(lon)::NUMERIC, 2) as min_longitude,
            ROUND(MAX(lon)::NUMERIC, 2) as max_longitude,
            ROUND(MIN(lat)::NUMERIC, 2) as min_latitude,
            ROUND(MAX(lat)::NUMERIC, 2) as max_latitude
        FROM archaeological_sites
    """).df()
    print(result)

def quick_spatial_sample(con, center_lat=31.7683, center_lon=35.2137, radius_km=50):
    """Get a sample of sites near a point"""
    
    print(f"\n=== Sample: Sites within {radius_km}km of ({center_lat}, {center_lon}) ===")
    result = con.execute("""
        SELECT 
            label,
            ROUND(lat::NUMERIC, 4) as lat,
            ROUND(lon::NUMERIC, 4) as lon,
            ROUND((ST_Distance(
                geom,
                ST_Point(?, ?)
            ) / 1000)::NUMERIC, 2) as distance_km
        FROM archaeological_sites
        WHERE ST_Distance(
            geom,
            ST_Point(?, ?)
        ) <= ?
        ORDER BY distance_km
        LIMIT 5
    """, [center_lon, center_lat, center_lon, center_lat, radius_km * 1000]).df()
    print(result)

def quick_density_analysis(con, grid_size=1.0):
    """Show site density in a coarse grid"""
    
    print(f"\n=== Site Density Hotspots (Grid Size: {grid_size}°) ===")
    result = con.execute("""
        WITH grid AS (
            SELECT 
                FLOOR(lon / ?) * ? as grid_lon,
                FLOOR(lat / ?) * ? as grid_lat
            FROM archaeological_sites
        )
        SELECT 
            grid_lon, 
            grid_lat,
            COUNT(*) as site_count
        FROM grid
        GROUP BY grid_lon, grid_lat
        ORDER BY site_count DESC
        LIMIT 5
    """, [grid_size, grid_size, grid_size, grid_size]).df()
    print(result)

# Optional: More intensive analyses
def analyze_clusters(con, sample_size=10000, distance_km=10):
    """Analyze clusters using a sample of the data"""
    
    print(f"\n=== Finding site clusters (using {sample_size} sample size) ===")
    result = con.execute("""
        WITH sample AS (
            SELECT *
            FROM archaeological_sites
            ORDER BY random()
            LIMIT ?
        )
        SELECT 
            a1.label as site,
            COUNT(*) as nearby_sites
        FROM sample a1
        JOIN sample a2
        ON ST_Distance(a1.geom, a2.geom) <= ?
        AND a1.uuid != a2.uuid
        GROUP BY a1.label
        ORDER BY nearby_sites DESC
        LIMIT 5
    """, [sample_size, distance_km * 1000]).df()
    print(result)

# Usage example:

# Initialize and load data
con = setup_duckdb()

# Create spatial table from your parquet file
parquet_file = P.home() / 'data/iSample/opencontext' / 'output_geoparquet.parquet'
create_spatial_table(con, parquet_file)


# Quick analyses
quick_stats(con)
quick_spatial_sample(con)
quick_density_analysis(con)

# Optional: Run cluster analysis on a sample
# analyze_clusters(con, sample_size=10000)


# Folium integration

In [ ]:
gdf.dtypes

In [ ]:
# Take a random sample of 1000 rows
sample_size = 1000
gdf_sample = gdf.sample(n=sample_size, random_state=42)

# Convert bytes to strings, handling encoding errors
def safe_decode(x):
    if isinstance(x, bytes):
        try:
            return x.decode('utf-8')
        except UnicodeDecodeError:
            try:
                return x.decode('latin-1')
            except:
                return str(x)
    return x

# Convert datetime columns to string and handle bytes in object columns
for col in gdf_sample.columns:
    if gdf_sample[col].dtype == 'object':
        gdf_sample[col] = gdf_sample[col].apply(safe_decode)
    elif 'datetime' in str(gdf_sample[col].dtype):
        gdf_sample[col] = gdf_sample[col].astype(str)

# Create a folium map centered on the mean coordinates of your geometries
center = [gdf_sample.geometry.centroid.y.mean(), gdf_sample.geometry.centroid.x.mean()]
m = folium.Map(location=center, zoom_start=4)

# Add the GeoDataFrame to the map
folium.GeoJson(
    gdf_sample,
    name='geojson',
    style_function=lambda x: {
        'fillColor': 'blue',
        'color': 'black',
        'weight': 1,
        'fillOpacity': 0.5
    }
).add_to(m)

# Add a layer control
folium.LayerControl().add_to(m)

# Display the map
m